In [228]:
import pandas as pd
pd.set_option('display.width', 1000)


from bs4 import BeautifulSoup
import requests

import re

## 1) get links to all M.S. thesis

In [49]:
raw_html = requests.get("https://library.columbia.edu/locations/journalism/masters.html").content
soup = BeautifulSoup(raw_html, "html.parser")
links = soup.find_all("ul",attrs={"class": "dropdown-menu"})[6].find_all("a", href=True)

In [54]:
year = 1995
link_list = []

for link in links:
    link_dict = {}
    
    url = "https://library.columbia.edu/" + link["href"]
    link_dict.update({
            "year" : year,
            "url" : url,
        })
    link_list.append(link_dict)
    
    year = year + 1

In [56]:
df_link = pd.DataFrame(link_list)

In [59]:
df_link.to_csv("master_project_links.csv", index = False)

## 2) scrape content, clean it, add into dictionary and list, turn into dataframe

In [539]:
## 2017

def get_2017(url):
    info_list = []
    raw_html = requests.get("https://library.columbia.edu/locations/journalism/masters/2017MastersProjects.html").content
    soup = BeautifulSoup(raw_html, "html.parser")
    everything = soup.find("div", attrs = {"id" : "primary"}).find_all("p")
    for thing in everything[1:]:
        info = thing.text
        info_dict = {}

        if "AUTHOR:" in info:
            author = re.findall("Author:(.*)", info, re.IGNORECASE)[0]
        elif "TITLE:" in info:
            title = re.findall("Title:(.*)", info, re.IGNORECASE)[0]
        elif "ADVISOR:" in info:
            advisor = re.findall("Advisor:(.*)", info, re.IGNORECASE)[0]
        elif "FORMAT:" in info:    
            format_ = re.findall("Format:(.*)", info, re.IGNORECASE)[0]

            info_dict.update({
                "author" : author,
                "title" : title,
                "advisor" : advisor,
                "project_format" : format_

            })
            info_list.append(info_dict)

    return info_list

In [398]:
## 1995 - 2012
def get_1995_2011(url):

    info_list = []
    raw_html = requests.get(url).content
    soup = BeautifulSoup(raw_html, "html.parser")
    everything = soup.find_all("div", attrs = {"class" : "cul-text" })
    for thing in everything[1:]:
        all_info = thing.text.split("\n\n")
        for info in all_info:
            info_dict = {}
            try:
                try:
                    author = re.findall("Author:(.*)", info)[0]
                except:
                    author = re.findall("Authors:(.*)", info)[0]
                title = re.findall("Title:(.*)", info)[0]
                advisor = re.findall("Advisor:(.*)", info)[0]

                format_ = re.findall("Format:(.*)", info)[0]


                info_dict.update({
                    "author" : author,
                    "title" : title,
                    "advisor" : advisor,
                    "project_format" : format_

                })

                info_list.append(info_dict)

            except:
                pass

    return info_list

In [540]:
## 2012 - 2016
def get_2012_2016(url):
    info_list = []
    raw_html = requests.get(url).content
    soup = BeautifulSoup(raw_html, "html.parser")
    everything = soup.find("div", attrs = {"id" : "primary"}).find_all("p")
    for thing in everything:
        info = thing.text
        info_dict = {}


        try:
            author = re.findall("Author:(.*)", info, re.IGNORECASE)[0]
        except:
            pass
        try:
            title = re.findall("Title:(.*)", info, re.IGNORECASE)[0]
        except:
            pass
        try:
            advisor = re.findall("Advisor:(.*)", info, re.IGNORECASE)[0]
        except:
            pass
        try:
            format_ = re.findall("Format:(.*)", info, re.IGNORECASE)[0]
        except:
            pass

        try:
            info_dict.update({
                "author" : author,
                "title" : title,
                "advisor" : advisor,
                "project_format" : format_

            })
            info_list.append(info_dict)

        except:
            pass

    return info_list


In [544]:
def get_dfs_test(row):
    if row["year"] == 2017:
        print(row["year"])
        df_temp = pd.DataFrame(get_2017(row["url"]))
        df_temp["year"] = row["year"]
        csv_name = "masters_project_" + str(row["year"]) + ".csv"
        df_temp.to_csv(csv_name, index = False)
        print(df_temp.shape)
        print("")     
    elif row["year"] == 2009 or row["year"] > 2011:
        print(row["year"])
        df_temp = pd.DataFrame(get_2012_2016(row["url"]))
        df_temp["year"] = row["year"]
        csv_name = "masters_project_" + str(row["year"]) + ".csv"
        df_temp.to_csv(csv_name, index = False)
        print(df_temp.shape)
        print("")        
        
    elif row["year"] < 2012:
        print(row["year"])
        df_temp = pd.DataFrame(get_1995_2011(row["url"]))
        df_temp["year"] = row["year"]
        csv_name = "masters_project_" + str(row["year"]) + ".csv"
        df_temp.to_csv(csv_name, index = False)
        print(df_temp.shape)
        print("")
        

In [545]:
df_link.apply(get_dfs_test, axis=1)

1995
(182, 5)

1996
(184, 5)

1997
(210, 5)

1998
(258, 5)

1999
(238, 5)

2000
(239, 5)

2001
(249, 5)

2002
(253, 5)

2003
(261, 5)

2004
(233, 5)

2005
(252, 5)

2006
(250, 5)

2007
(250, 5)

2008
(270, 5)

2009
(218, 5)

2010
(247, 5)

2011
(184, 5)

2012
(294, 5)

2013
(266, 5)

2014
(215, 5)

2015
(240, 5)

2016
(197, 5)

2017
(224, 5)



0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
dtype: object

## Now merge all csv

In [546]:
import glob

In [548]:
df_all = pd.concat([pd.read_csv(f) for f in glob.glob('masters_project_*.csv')], ignore_index = True)

In [2]:
# df_all.shape

In [1]:
# df_all.head()